In [0]:
pip install syft

In [0]:
import sys
import torch as torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F
import syft as sy

hook = sy.TorchHook(torch)



In [2]:
x = torch.tensor([1,2,3,4,5])
y = x+x
print(y)

tensor([ 2,  4,  6,  8, 10])


Sending tensor to jake's m/c

In [3]:
jake = sy.VirtualWorker(hook, id="jake")
print("Jake has: " + str(jake._objects))

x = torch.tensor([1,2,3,4,5])
y = torch.ones(5)

x_ptr = x.send(jake)  # pointer to tensor on jake's device
y_ptr = y.send(jake)

print("Jake has: " + str(jake._objects))

Jake has: {}
Jake has: {21304716689: tensor([1, 2, 3, 4, 5]), 69143685298: tensor([1., 1., 1., 1., 1.])}


In [4]:
z = x_ptr + x_ptr
z

(Wrapper)>[PointerTensor | me:4685883561 -> jake:26726461718]

In [5]:
print(x_ptr.location, "\nJake has: " + str(jake._objects))

<VirtualWorker id:jake #objects:3> 
Jake has: {21304716689: tensor([1, 2, 3, 4, 5]), 69143685298: tensor([1., 1., 1., 1., 1.]), 26726461718: tensor([ 2,  4,  6,  8, 10])}


In [6]:
jake

<VirtualWorker id:jake #objects:3>

In [7]:
jake == x_ptr.location

True

In [8]:
x_ptr.id_at_location

21304716689

In [9]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [0]:
me = sy.local_worker


In [11]:
x_ptr

(Wrapper)>[PointerTensor | me:31339515241 -> jake:21304716689]

In [12]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [13]:
y_ptr

(Wrapper)>[PointerTensor | me:11084576579 -> jake:69143685298]

In [14]:
y_ptr.get()

tensor([1., 1., 1., 1., 1.])

In [15]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [16]:
print("Jake has: " + str(jake._objects))

Jake has: {}


 Commands will be serialized and sent to jake, who will perform the computation, create a tensor, and then returned the pointer to z back to us!

If we call .get() on the pointer, we will then receive the result back to our machine!

In [18]:
x = torch.tensor([1,2,3,4,5]).send(jake)
y = torch.tensor([1,1,1,1,1]).send(jake)

z = x + y

print(z)

(Wrapper)>[PointerTensor | me:52496977762 -> jake:26922698724]


In [19]:
z.get()

tensor([2, 3, 4, 5, 6])

In [20]:
# Torch fns:-

z = torch.add(x,y)
z.get()

tensor([2, 3, 4, 5, 6])

In [21]:
# Back prop:-

x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(jake)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(jake)

z = (x + y).sum()
z

(Wrapper)>[PointerTensor | me:41313182917 -> jake:68642700085]

In [22]:
z.backward()

(Wrapper)>[PointerTensor | me:64386017395 -> jake:78675884520]

In [23]:
x.grad

(Wrapper)>[PointerTensor | me:13663044548 -> jake:32769583512]::grad

In [0]:
# x = x.get()
# x.grad